# Prompt Engineering

1. Set a goal for the prompt.

    **Ex:** *Write a prompt that generates a 1-day meal plan for an athelete based upon their hight, weight, goal and dietary restrictions.*
2. Write an initial prompt.
3. Eval the prompt.
4. Apply a prompt engineering technique.
5. Re-eval to verify better performance .
6. repeat 4 & 5 until satisfied with performance.


*Note – New classes were created for this notebook:*
- ClaudeResponsesClass: functions to chat with Claude.
- PromptEvaluatorClass: functions to evaluate responses by a model.

#### Import classes and set constants

In [5]:
from lib.PromptEvaluatorClass import PromptEvaluator
from lib.ClaudeResponsesClass import ClaudeResponses

In [6]:
# Create an instance of PromptEvaluator
evaluator = PromptEvaluator(max_concurrent_tasks=3)
# Create an instance of ClaudeResponses
claude = ClaudeResponses(save_history=False)

#### Generate sample dataset

In [ ]:
dataset_file = "03_PromptEngineering_outputs/sample_dataset.json"
dataset = evaluator.generate_dataset(
    # Describe the purpose of the prompt you're trying to test
    task_description="Write a compact, concise 1 day meal plan for a single athlete",
    # Decribe the different imputs that your prompt requires
    prompt_inputs_spec={
        "height": "Athlete's height in cm",
        "weight": "Athlete's weight in kg", 
        "goal": "Goal of the athlete",
        "restrictions": "Dietary restrictions of the athlete"
    },
    # Where to write the generated dataset
    output_file=dataset_file,
    # Number of test cases to generate
    num_cases=3,
    
)

Generated 1/3 test cases
Generated 2/3 test cases
Generated 3/3 test cases


#### Simple prompt

In [ ]:
# Function containing prompt
def run_prompt(prompt_inputs):
    prompt = f"""
    What should this person eat?

    - Height: {prompt_inputs["height"]}
    - Weight: {prompt_inputs["weight"]}
    - Goal: {prompt_inputs["goal"]}
    - Dietary restrictions: {prompt_inputs["restrictions"]}
    """
    return claude.chat(prompt)

In [ ]:
extra_criteria = """
The output should include:
- Daily caloric total
- Macronutrient breakdown  
- Meals with exact foods, portions, and timing
"""
results = evaluator.run_evaluation(
    run_prompt_function=run_prompt,
    dataset_file="03_PromptEngineering_outputs/sample_dataset.json",
    extra_criteria=extra_criteria,
json_output_file="03_PromptEngineering_outputs/sample_json_output.json",
html_output_file="03_PromptEngineering_outputs/sample_html_output.html"
)

Graded 1/3 test cases
Graded 2/3 test cases
Graded 3/3 test cases
Average score: 3


## 1. Be clear and direct

First line of prompt: tends to be the most important. Set the action and provide the task; set basic expectations for the output.

**Clear**
- Use simple language
- State what you want explicitly.
- Lead your prompt with a simple statement of the model's task.

**Direct**
- Use instructions, not questions.
- Use direct action verbs (write, create, generate, ...)

In [13]:
def run_prompt(prompt_inputs):
    # Direct action verb at the start
    # Direct task in simple language
    prompt = f"""
    Generate a one-day meal plan for an athlete that meets their dietary restrictions.

    - Height: {prompt_inputs["height"]}
    - Weight: {prompt_inputs["weight"]}
    - Goal: {prompt_inputs["goal"]}
    - Dietary restrictions: {prompt_inputs["restrictions"]}
    """
    return claude.chat(prompt)

In [18]:
def eval_prompt(run_prompt_function, file):
    results = evaluator.run_evaluation(
    run_prompt_function=run_prompt_function,
    dataset_file=dataset_file,
    extra_criteria=extra_criteria,
    json_output_file=f"03_PromptEngineering_outputs/{file}.json",
    html_output_file=f"03_PromptEngineering_outputs/{file}.html"
    )
    return results

In [ ]:
file = "clear_direct_prompt"

results = eval_prompt(run_prompt, file)

Graded 1/3 test cases
Graded 2/3 test cases
Graded 3/3 test cases
Average score: 6


## 2. Be Specific

Provide clead guidelines/steps that direct Claude to the desired output.
- List qualities that output should have: length of the response, structure and format, specific attributes or elements to inclure, tone or style requirements. 
    - Use in practically every prompt.
- Provide steps the model should follow: steps that will help it think through a problem systematically or consider multiple perspectives.
    - Use to troubleshoot problems, decision-making, critical thinking, or any situation to consider multiple angles.

#### Guidelines

In [ ]:
def run_prompt(prompt_inputs):
    # Direct action verb at the start
    # Direct task in simple language
    prompt = f"""
    Generate a one-day meal plan for an athlete that meets their dietary restrictions.

    - Height: {prompt_inputs["height"]}
    - Weight: {prompt_inputs["weight"]}
    - Goal: {prompt_inputs["goal"]}
    - Dietary restrictions: {prompt_inputs["restrictions"]}

    Guidelines:
    1. Include accurate daily calorie amount
    2. Show protein, fat, and carb amounts  
    3. Specify when to eat each meal
    4. Use only foods that fit restrictions
    5. List all portion sizes in grams
    6. Keep budget-friendly if mentioned
    """
    return claude.chat(prompt)

In [26]:
file = "guidelines_prompt"

results = eval_prompt(run_prompt, file)

Graded 1/3 test cases
Graded 2/3 test cases
Graded 3/3 test cases
Average score: 6.333333333333333


#### Steps

In [27]:
def run_prompt(prompt_inputs):
    # Direct action verb at the start
    # Direct task in simple language
    prompt = f"""
    Generate a one-day meal plan for an athlete that meets their dietary restrictions.

    - Height: {prompt_inputs["height"]}
    - Weight: {prompt_inputs["weight"]}
    - Goal: {prompt_inputs["goal"]}
    - Dietary restrictions: {prompt_inputs["restrictions"]}

    Follow these steps:
    1. Calculate daily calories needed
    2. Figure out protein, fat, carb amounts
    3. Plan meal timing around workouts
    4. Choose foods that fit restrictions
    5. set portion sizes in grams
    6. Adjust for budget if needed
    """
    return claude.chat(prompt)

In [28]:
file = "steps_prompt"

results = eval_prompt(run_prompt, file)

Graded 1/3 test cases
Graded 2/3 test cases
Graded 3/3 test cases
Average score: 8.666666666666666


## 3. Structure with XML tags and Provide examples

Use **XML** tags to separate distinct portions of the prompt.
- Most useful when including a lot of context.
- Help serve as delimiters for the model.

Give the model sample input/output pairs.
- Useful for capturing corner cases or complex output formats
- *"One-Shot*: provide a single example
- *"Multi-Shot*: provide multiple examples
- Combine with XML Tags for structure

In [ ]:
def run_prompt(prompt_inputs):
    prompt = f"""
    Generate a one-day meal plan for an athlete that meets their dietary restrictions.

    <athlete_information>
    - Height: {prompt_inputs["height"]}
    - Weight: {prompt_inputs["weight"]}
    - Goal: {prompt_inputs["goal"]}
    - Dietary restrictions: {prompt_inputs["restrictions"]}
    </athlete_information>
    
    Follow these steps:
    1. Calculate daily calories needed
    2. Figure out protein, fat, carb amounts
    3. Plan meal timing around workouts
    4. Choose foods that fit restrictions
    5. set portion sizes in grams
    6. Adjust for budget if needed

    Here's an example with a sample input and an ideal output:
    SAMPLE INPUT:
    <sample_input>
    - Height: 180
    - Weight: 85
    - Goal: muscle maintenance and performance optimization during training season
    - Restrictions: lactose intolerant, limited budget for groceries
    </sample_input>

    IDEAL OUTPUT:
    <ideal_output>
    1. Daily Calorie Calculation:
    - Assuming moderate to high activity level
    - Basal Metabolic Rate (BMR) using Mifflin-St Jeor Equation:
        BMR = (10 × weight) + (6.25 × height) - (5 × age) + 5
        Assuming age 25: BMR ≈ 1,925 calories
    - Activity multiplier (training season): 1.7
    - Total Daily Energy Expenditure (TDEE): 1,925 × 1.7 = 3,270 calories

    2. Macronutrient Breakdown:
    - Protein: 2.2g per kg of body weight
        85 kg × 2.2 = 187g protein (748 calories)
    - Carbohydrates: 5-7g per kg of body weight
        85 kg × 6 = 510g carbs (2,040 calories)
    - Fats: Remaining calories
        3,270 - (748 + 2,040) = 482 calories (54g fat)

    3. Meal Timing:
    - Pre-workout meal: Complex carbs, lean protein
    - Post-workout meal: High protein, quick-absorbing carbs
    - Meals spread 3-4 hours apart

    4. Lactose-Free, Budget-Friendly Meal Plan:

    Breakfast (7:00 AM) - 700 calories:
    - Oatmeal (100g dry)
    - Plant-based protein powder (30g)
    - Banana (100g)
    - Almond butter (15g)
    - Chia seeds (10g)

    Mid-Morning Snack (10:00 AM) - 350 calories:
    - Rice cakes (50g)
    - Canned tuna (100g)
    - Avocado (50g)

    Lunch (1:00 PM) - 800 calories:
    - Chicken breast (150g)
    - Brown rice (100g)
    - Mixed vegetables (200g)
    - Olive oil (15ml)

    Pre-Workout Snack (4:00 PM) - 400 calories:
    - Sweet potato (150g)
    - Turkey breast (100g)
    - Spinach (50g)

    Post-Workout Meal (6:00 PM) - 700 calories:
    - Lean ground beef (120g)
    - Quinoa (100g)
    - Black beans (100g)
    - Mixed vegetables (150g)

    Evening Snack (9:00 PM) - 320 calories:
    - Egg whites (100g)
    - Oats (50g)
    - Berries (100g)

    5. Nutritional Breakdown:
    - Total Calories: 3,270
    - Protein: 187g
    - Carbohydrates: 510g
    - Fat: 54g

    6. Budget Considerations:
    - Bulk buying proteins (chicken, ground beef)
    - Frozen vegetables
    - Canned proteins
    - Buying in-season produce
    - Buying larger packages of staples like rice, oats

    Additional Notes:
    - Drink 3-4 liters of water daily
    - Use seasonings and herbs for flavor
    - Adjust portions if needed based on individual response

    Supplements to Consider (Optional):
    - Lactose-free protein powder
    - Multivitamin
    - Creatine monohydrate

    Recommendations:
    - Meal prep to save time and money
    - Rotate protein sources for variety
    - Monitor body's response and adjust as needed
    </ideal_output>

    The sample output thoroughly meets all mandatory requirements by providing a precise daily meal plan with exact caloric intake (3,270 calories), detailed macronutrient breakdown, specific meals with portions and timing, and accommodations for lactose intolerance. The plan demonstrates careful nutritional engineering for an athlete's performance needs while respecting budget constraints.
    """
    return claude.chat(prompt)

In [33]:
file = "xml_tags_examples"

results = eval_prompt(run_prompt, file)

Graded 1/3 test cases
Graded 2/3 test cases
Graded 3/3 test cases
Average score: 8.333333333333334


## Exercise

Goal: improve an existing prompt to take in a passage of text and extract topics into a JSON array.

In [34]:
dataset_file = "03_PromptEngineering_outputs/exercise_dataset.json"
dataset = evaluator.generate_dataset(
    task_description="Extract topics out of a passage of text from a scholarly article into a JSON array of strings",
    prompt_inputs_spec={"content": "One paragraph of text from a scholarly journal written in English"},
    output_file=dataset_file,
    num_cases=5
)

Generated 1/5 test cases
Generated 2/5 test cases
Generated 3/5 test cases
Generated 4/5 test cases
Generated 5/5 test cases


In [39]:
def run_prompt(prompt_inputs):
    prompt = f"""
    Extract key topics mentioned from a passage of text from a scholarly journal into a JSON array of strings.

    <text>
    {prompt_inputs["content"]}
    </text> 

    Follow these steps:
    1. Closely examine the provided text
    2. Identify each topic mentioned
    3. Add each topic to a JSON array
    4. Respond with the JSON array. Do not provide any other text or commentary.
    """
    return claude.chat(prompt)
    

In [40]:
file = "exercise"
extra_criteria = """
- Contains a JSON array of strings, containing each topic mentioned in the article.
- The strings should contain only a topic without any extra commentary.
- Response should contain the JSON array and nothing else.
"""

results = eval_prompt(run_prompt, file)

Graded 1/5 test cases
Graded 2/5 test cases
Graded 3/5 test cases
Graded 4/5 test cases
Graded 5/5 test cases
Average score: 9
